# Running CHEASE actor

In [ ]:
import Pkg
#println(joinpath(dirname(abspath(__@file__)),"..",".."))
Pkg.activate("../..")
using Revise
using FUSE
using Plots; gr();
global_logger(FUSE.logger);

In [ ]:
dd, ini, act = FUSE.init(:ITER,init_from=:ods);

### Initialize the ITER case from ODS
[ITER case documentation](https://fuse.help/cases.html#ITER)
#### Let's run the ActorCHEASE on the ITER equilibrium from the ods and compare 

[chease actor](https://fuse.help/actors.html#CHEASE)

In [ ]:
function smooth_rz_bound_MXH(eqt::IMAS.equilibrium__time_slice,order)
    eqt_bound = eqt.boundary.outline
    mxh = IMAS.MXH(eqt_bound.r,eqt_bound.z,order)

    rz_array = mxh()
    eqt_bound.r = rz_array[1]
    eqt_bound.z = rz_array[2]
end

dd, ini, act = FUSE.init(:ITER,init_from=:ods);
eq_plot = plot(dd.equilibrium, label="original ITER eq")

#order = 3

#smooth_rz_bound_MXH(dd.equilibrium.time_slice[],order)
#display(plot!(eq_plot,dd.equilibrium.time_slice[].boundary.outline.r,dd.equilibrium.time_slice[].boundary.outline.z,label="MXH boundary given", lw=4,ls=:dash))
actor = FUSE.ActorCHEASE(dd,act);

display(plot!(eq_plot,dd.equilibrium, label="CHEASE's ITER eq order = $order", psi_levels_inside=[-1]))



In [ ]:
eq_plot = plot(dd_org.equilibrium, label="original ITER eq")
plot!(eq_plot,dd.equilibrium, psi_levels_in=[0],lw=2,label="CHEASE output",lcfs=false)
display(plot!(eq_plot,dd.equilibrium.time_slice[].boundary.outline.r,dd.equilibrium.time_slice[].boundary.outline.z,label="MXH boundary given", lw=4,ls=:dash))
wall_outline = dd.wall.description_2d[1].limiter.unit[1].outline

plot!(wall_outline.r, wall_outline.z,lw=3,label="wall")

### Running CHEASE on kinetic profiles and steady state current
To do this we need to do a transport step followed by the calculation of the steady-state current

#### Transport

In [ ]:
dd, ini, act = FUSE.init(:ITER,init_from=:ods);

core_profiles_plot = plot(dd.core_profiles)
FUSE.ActorSteadyStateCurrent(dd,act)
FUSE.ActorTauenn(dd,act)
plot!(core_profiles_plot,dd.core_profiles)



#### Steady-state current & running CHEASE using kinetic profiles and ss current

In [ ]:
FUSE.ActorSteadyStateCurrent(dd,act);

act.ActorCHEASE.j_tor_from = :core_profiles 
act.ActorCHEASE.pressure_from = :core_profiles

order=3
eq_plot = plot(dd.equilibrium, label="original ITER eq")
q_plot = plot(dd.equilibrium.time_slice[].profiles_1d.rho_tor_norm,dd.equilibrium.time_slice[].profiles_1d.q,label="original q-profile")

smooth_rz_bound_MXH(dd.equilibrium.time_slice[],order)

actor = FUSE.ActorCHEASE(dd,act);
display(plot!(eq_plot,dd.equilibrium, label="CHEASE's ITER eq with transport and SS current"))
#display(plot!(q_plot,dd.equilibrium.time_slice[].profiles_1d.rho_tor_norm, -1.* dd.equilibrium.time_slice[].profiles_1d.q,label="CHEASE q-profile"))


### Similar analysis for a D3D shot

In [ ]:
for order in [1,3,6,9]

    dd, ini, act = FUSE.init(:D3D)

    eq_plot = plot(dd.equilibrium, label="original D3D eq")

    smooth_rz_bound_MXH(dd.equilibrium.time_slice[],order)


    actor = FUSE.ActorCHEASE(dd,act);
    display(plot!(dd.equilibrium.time_slice[].boundary.outline.r,dd.equilibrium.time_slice[].boundary.outline.z,label="MXH boundary given", lw=4,ls=:dash))

    display(plot!(eq_plot,dd.equilibrium, label="CHEASE's D3D eq order = $order",lw=2))
#    display(plot!(q_plot,dd.equilibrium.time_slice[].profiles_1d.rho_tor_norm,dd.equilibrium.time_slice[].profiles_1d.q,label="CHEASE q-profile"))
end

### D3D transport & steady-state current

In [ ]:
dd, ini, act = FUSE.init(:D3D);

core_profiles_plot = plot(dd.core_profiles)
FUSE.ActorSteadyStateCurrent(dd,act)
FUSE.ActorTauenn(dd,act)
display(plot!(core_profiles_plot,dd.core_profiles))

FUSE.ActorSteadyStateCurrent(dd,act);

act.ActorCHEASE.j_tor_from = :core_profiles 
act.ActorCHEASE.pressure_from = :core_profiles

eq_plot = plot(dd.equilibrium, label="original ITER eq")
q_plot = plot(dd.equilibrium.time_slice[].profiles_1d.rho_tor_norm,dd.equilibrium.time_slice[].profiles_1d.q,label="original q-profile")

actor = FUSE.ActorCHEASE(dd,act);
display(plot!(eq_plot,dd.equilibrium, label="CHEASE's ITER eq with transport and SS current"))
display(plot!(q_plot,dd.equilibrium.time_slice[].profiles_1d.rho_tor_norm,dd.equilibrium.time_slice[].profiles_1d.q,label="CHEASE q-profile"))



In [ ]:
dd, ini, act = FUSE.init(:D3D);
eqtbd= dd.equilibrium.time_slice[].boundary.outline

display(plot(dd.equilibrium))
for order in [1,3,6,9]

    mxh = IMAS.MXH(eqtbd.r,eqtbd.z,3)

    rz_array = mxh()
    r_new = []
    z_new = []
    for (i,j) in rz_array
        println("$i , $j")
        r_new = vcat(r_new,i)
        z_new = vcat(z_new,j)
    end
    @show typeof(r_new)
    #curvature = IMAS.curvature(r_new,z_new)
    #@show r_new, z_new
    #IMAS.curvature(
    plot(dd.equilibrium,label="orig eq, $(round(maximum(IMAS.curvature(eqtbd.r,eqtbd.z)),digits=2))")
#    display(plot!(r_new, z_new,label="$(order)",lw=5))
    
    r_newx = [float(i) for i in r_new]
    z_newx = [float(i) for i in z_new]
    push!(r_newx,r_newx[1])
    push!(z_newx,z_newx[1])

    max_curv = maximum(IMAS.curvature(r_newx,z_newx))
    @show max_curv
    display(plot!(r_new, z_new,label="$(order) max curvature = $(round(max_curv,digits=2))",lw=5))
end




In [ ]:
dd, ini, act = FUSE.init(:D3D);

eqt = dd.equilibrium.time_slice[]
function smooth_rz_bound_MXH(eqt::IMAS.equilibrium__time_slice,order)
    eqt_bound = eqt.boundary.outline
    mxh = IMAS.MXH(eqt_bound.r,eqt_bound.z,order)

    rz_array = mxh()
    eqt_bound.r = rz_array[1]
    eqt_bound.z = rz_array[2]
end

smooth_rz_bound_MXH(eqt,3)



In [ ]:
dd, ini, act = FUSE.init(:FPP, version=:v1_demount,init_from=:ods);
eq_plot = plot(dd.equilibrium, label="original ITER eq")
#q_plot = plot(dd.equilibrium.time_slice[].profiles_1d.rho_tor_norm,dd.equilibrium.time_slice[].profiles_1d.q,label="original q-profile")

smooth_rz_bound_MXH(dd.equilibrium.time_slice[],1)

display(plot!(dd.equilibrium.time_slice[].boundary.outline.r,dd.equilibrium.time_slice[].boundary.outline.z))
actor = FUSE.ActorCHEASE(dd,act);
display(plot!(eq_plot,dd.equilibrium, label="CHEASE's ITER eq"))
#display(plot!(q_plot,dd.equilibrium.time_slice[].profiles_1d.rho_tor_norm, -1 .* dd.equilibrium.time_slice[].profiles_1d.q,label="CHEASE q-profile"))
